In [ ]:
import gdstk
import numpy as np

In [ ]:
WAFER_DIAMETER = 150000

FIELD_SIZE = 300

# hall bar shape
HALL_BAR_SIZE_X = 10
HALL_BAR_SIZE_Y = 10
HALL_BAR_OFFSET_X = 0
HALL_BAR_OFFSET_Y = 0
HALL_BAR_OFFSET_R = 0

# metal contact shapes
METAL_CONTACT_WIDTH = 0.6
METAL_CONTACT_OVERLAP = 1
METAL_CONTACT_MATING_SIZE = 5
METAL_PAD_SIZE = 250

# total contact count is METAL_CONTACT_SIDES * METAL_CONTACT_COUNT * 2
METAL_CONTACT_SIDES = 4
METAL_CONTACT_COUNT = 2

# dicing blade dimensions
BLADE_KERF = 100
CHIP_BORDER_WIDTH = 500

In [ ]:
METAL_CONTACT_PITCH_X = HALL_BAR_SIZE_X / (METAL_CONTACT_COUNT + 1) * 0.5
METAL_CONTACT_PITCH_Y = HALL_BAR_SIZE_Y / (METAL_CONTACT_COUNT + 1) * 0.5

ACTIVE_REGION_SIZE = (
    np.ceil((METAL_CONTACT_COUNT + 1) * METAL_PAD_SIZE * 8 / 1000) * 1000
)

CHIP_SIZE = ACTIVE_REGION_SIZE + 4000
print(CHIP_SIZE)

In [ ]:
lib = gdstk.Library(unit=1e-6, precision=1e-9)
cell_chip = lib.new_cell("CHIP")

In [ ]:
# chip metal border

outer = CHIP_SIZE * 0.5 - BLADE_KERF * 0.5
inner = CHIP_SIZE * 0.5 - BLADE_KERF * 0.5 - CHIP_BORDER_WIDTH
cell_chip.add(
    *gdstk.boolean(
        gdstk.rectangle((-outer, -outer), (outer, outer)),
        gdstk.rectangle((-inner, -inner), (inner, inner)),
        "not",
        layer=0,
        datatype=0,
    )
)

In [ ]:
# CABL REG2 automatic alignment mark

cell_cabl_mark_reg2 = lib.new_cell("CABL_MARK_REG2")
cell_cabl_mark_reg2.add(gdstk.rectangle((-300, -1.5), (300, 1.5), layer=0, datatype=0))
cell_cabl_mark_reg2.add(gdstk.rectangle((-1.5, -300), (1.5, 300), layer=0, datatype=0))

cell_chip.add(
    gdstk.Reference(
        cell_cabl_mark_reg2,
        (-(CHIP_SIZE * 0.5 - 1000), -(CHIP_SIZE * 0.5 - 1000)),
    )
)
cell_chip.add(
    gdstk.Reference(
        cell_cabl_mark_reg2,
        (-(CHIP_SIZE * 0.5 - 1000), (CHIP_SIZE * 0.5 - 1000)),
    )
)

In [ ]:
# MLA150 automatic alignment mark

cell_mla_mark = lib.new_cell("MLA_MARK")
cell_mla_mark_poly0 = gdstk.Polygon(
    [
        (-0.75, -0.75),
        (-0.75, -7.5),
        (-125, -7.5),
        (-125, -15),
        (-425, -15),
        (-425, 15),
        (-125, 15),
        (-125, 7.5),
        (-0.75, 7.5),
        (-0.75, 0.75),
        (-7.5, 0.75),
        (-7.5, -0.75),
    ],
    layer=0,
    datatype=0,
)
cell_mla_mark.add(cell_mla_mark_poly0)
cell_mla_mark.add(cell_mla_mark_poly0.copy().rotate(np.pi * 0.5))
cell_mla_mark.add(cell_mla_mark_poly0.copy().rotate(np.pi))
cell_mla_mark.add(cell_mla_mark_poly0.copy().rotate(np.pi * 1.5))

cell_chip.add(
    gdstk.Reference(
        cell_mla_mark, (-(CHIP_SIZE * 0.5 - 2000), -(CHIP_SIZE * 0.5 - 2000))
    )
)
cell_chip.add(
    gdstk.Reference(
        cell_mla_mark, (-(CHIP_SIZE * 0.5 - 2000), (CHIP_SIZE * 0.5 - 2000))
    )
)
cell_chip.add(
    gdstk.Reference(
        cell_mla_mark, ((CHIP_SIZE * 0.5 - 2000), -(CHIP_SIZE * 0.5 - 2000))
    )
)

In [ ]:
# Additional chip logos & markers

cell_chip.add(
    *gdstk.text(
        "Twisted Graphene Devices, Substrate v1\nDaniel He, Cao Lab, 2025",
        100,
        (-(CHIP_SIZE * 0.5 - 3000), -(CHIP_SIZE * 0.5 - 2000)),
        layer=0,
        datatype=0,
    )
)

lib_symbols = gdstk.read_gds("./gdslib_fun_symbols/main.gds")

lib.add(lib_symbols["CAL_LOGO"])
cell_chip.add(
    gdstk.Reference(
        lib_symbols["CAL_LOGO"],
        (-(CHIP_SIZE * 0.5 - 2800), -(CHIP_SIZE * 0.5 - 2000)),
        magnification=20,
    )
)

lib.add(lib_symbols["EYE_OF_THE_UNIVERSE"])
cell_chip.add(
    gdstk.Reference(
        lib_symbols["EYE_OF_THE_UNIVERSE"],
        ((CHIP_SIZE * 0.5 - 2000), (CHIP_SIZE * 0.5 - 2000)),
        magnification=100,
    )
)

In [ ]:
cell_sample_region_mark = lib.new_cell("SAMPLE_REGION_MARK")
cell_sample_region_mark.add(gdstk.rectangle((-4, -4), (0, 0), layer=0, datatype=0))
cell_sample_region_mark.add(gdstk.rectangle((0, 0), (2, 2), layer=0, datatype=0))

cell_chip.add(
    gdstk.Reference(
        cell_sample_region_mark,
        (-FIELD_SIZE * 0.125, -FIELD_SIZE * 0.125),
        rotation=0,
    )
)
cell_chip.add(
    gdstk.Reference(
        cell_sample_region_mark,
        (FIELD_SIZE * 0.125, -FIELD_SIZE * 0.125),
        rotation=np.pi * 0.5,
    )
)
cell_chip.add(
    gdstk.Reference(
        cell_sample_region_mark,
        (-FIELD_SIZE * 0.125, FIELD_SIZE * 0.125),
        rotation=np.pi * 1.5,
    )
)

In [ ]:
cell_metal_pad = lib.new_cell("METAL_PAD")

for i in range(METAL_CONTACT_COUNT):
    p0 = (
        -(ACTIVE_REGION_SIZE * 0.5 - METAL_PAD_SIZE * 2.5),
        -METAL_PAD_SIZE * 2 * (2 * i + 1),
    )
    p1 = (-FIELD_SIZE * 4, -METAL_PAD_SIZE * 2 * (2 * i + 1))
    p2 = (-FIELD_SIZE * 2, -METAL_CONTACT_MATING_SIZE * 4 * (2 * i + 1))
    p3 = (-FIELD_SIZE * 0.5, -METAL_CONTACT_MATING_SIZE * 4 * (2 * i + 1))

    trace = gdstk.FlexPath(p0, METAL_PAD_SIZE * 0.5)
    trace.bezier(
        [p0, p1, p2, (p3[0] + METAL_CONTACT_MATING_SIZE * 2, p3[1])],
        width=METAL_CONTACT_MATING_SIZE * 2,
    )
    cell_metal_pad.add(
        *gdstk.boolean(
            trace,
            gdstk.rectangle(
                (p0[0] - METAL_PAD_SIZE, p0[1] - METAL_PAD_SIZE),
                (p0[0] + METAL_PAD_SIZE, p0[1] + METAL_PAD_SIZE),
            ),
            "or",
            layer=0,
            datatype=0,
        )
    )

for s in range(METAL_CONTACT_SIDES):
    cell_chip.add(
        gdstk.Reference(
            cell_metal_pad, (0, 0), rotation=s * np.pi * 0.5, x_reflection=False
        )
    )
    cell_chip.add(
        gdstk.Reference(
            cell_metal_pad, (0, 0), rotation=s * np.pi * 0.5, x_reflection=True
        )
    )

In [ ]:
def rotate(point, angle):
    return (
        point[0] * np.cos(angle) - point[1] * np.sin(angle),
        point[0] * np.sin(angle) + point[1] * np.cos(angle),
    )


def translate_offset(point):
    return (
        point[0] * np.cos(HALL_BAR_OFFSET_R)
        - point[1] * np.sin(HALL_BAR_OFFSET_R)
        + HALL_BAR_OFFSET_X,
        point[0] * np.sin(HALL_BAR_OFFSET_R)
        + point[1] * np.cos(HALL_BAR_OFFSET_R)
        + HALL_BAR_OFFSET_Y,
    )


cell_hall_bar_etch = lib.new_cell("HALL_BAR_ETCH")
cell_hall_bar_etch.add(
    *gdstk.boolean(
        gdstk.rectangle(
            (-FIELD_SIZE * 0.5, -FIELD_SIZE * 0.5),
            (FIELD_SIZE * 0.5, FIELD_SIZE * 0.5),
        ),
        gdstk.Polygon(
            [
                translate_offset((-HALL_BAR_SIZE_X * 0.5, -HALL_BAR_SIZE_Y * 0.5)),
                translate_offset((HALL_BAR_SIZE_X * 0.5, -HALL_BAR_SIZE_Y * 0.5)),
                translate_offset((HALL_BAR_SIZE_X * 0.5, HALL_BAR_SIZE_Y * 0.5)),
                translate_offset((-HALL_BAR_SIZE_X * 0.5, HALL_BAR_SIZE_Y * 0.5)),
            ]
        ),
        "not",
        layer=1,
        datatype=0,
    )
)

cell_chip.add(gdstk.Reference(cell_hall_bar_etch, (0, 0)))


cell_metal_contact = lib.new_cell("METAL_CONTACT")

metal_trace_p0 = []
metal_trace_p1 = []
metal_trace_p2 = []
metal_trace_p3 = []
for s in range(METAL_CONTACT_SIDES):
    for i in range(METAL_CONTACT_COUNT):
        for sign in [-1, 1]:
            p0 = rotate(
                (-FIELD_SIZE * 0.5, sign * METAL_CONTACT_MATING_SIZE * 4 * (2 * i + 1)),
                s * np.pi * 0.5,
            )
            p1 = rotate(
                (-FIELD_SIZE * 0.3, sign * METAL_CONTACT_MATING_SIZE * 4 * (2 * i + 1)),
                s * np.pi * 0.5,
            )

            if s % 2 == 0:
                p2 = translate_offset(
                    rotate(
                        (-FIELD_SIZE * 0.15, sign * METAL_CONTACT_PITCH_Y * (i + 0.5)),
                        s * np.pi * 0.5,
                    )
                )
                p3 = translate_offset(
                    rotate(
                        (
                            -HALL_BAR_SIZE_X * 0.5 + METAL_CONTACT_OVERLAP,
                            sign * METAL_CONTACT_PITCH_Y * (i + 0.5),
                        ),
                        s * np.pi * 0.5,
                    )
                )
            else:
                p2 = translate_offset(
                    rotate(
                        (
                            -sign * METAL_CONTACT_PITCH_X * (i + 0.5),
                            -FIELD_SIZE * 0.15,
                        ),
                        (s - 1) * np.pi * 0.5,
                    )
                )
                p3 = translate_offset(
                    rotate(
                        (
                            -sign * METAL_CONTACT_PITCH_X * (i + 0.5),
                            -HALL_BAR_SIZE_Y * 0.5 + METAL_CONTACT_OVERLAP,
                        ),
                        (s - 1) * np.pi * 0.5,
                    )
                )

            trace = gdstk.FlexPath(p0, METAL_CONTACT_MATING_SIZE, layer=2, datatype=0)
            trace.bezier([p0, p1, p2, p3], width=METAL_CONTACT_WIDTH)
            cell_metal_contact.add(trace)

cell_chip.add(gdstk.Reference(cell_metal_contact, (0, 0)))

In [ ]:
cell_wafer = lib.new_cell("WAFER")

chip_array_size = int(WAFER_DIAMETER * 0.5 / CHIP_SIZE) * 2

chip_array = [
    [
        (
            CHIP_SIZE * (i - (chip_array_size - 1) * 0.5),
            CHIP_SIZE * (j - (chip_array_size - 1) * 0.5),
        )
        for i in range(chip_array_size)
    ]
    for j in range(chip_array_size)
]

chip_count = 0
for row in chip_array:
    for coord in row:
        flag = True
        for xsign in [-1, 1]:
            for ysign in [-1, 1]:
                if (coord[0] + xsign * CHIP_SIZE * 0.5) ** 2 + (
                    coord[1] + ysign * CHIP_SIZE * 0.5
                ) ** 2 >= (WAFER_DIAMETER * 0.5) ** 2:
                    flag = False
        if flag:
            chip_count += 1
            cell_wafer.add(gdstk.Reference(cell_chip, coord))

print(chip_count)

In [ ]:
lib.write_gds("main.gds")